In [5]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib
import os
import pymorphy2
import re
from copy import deepcopy
import operator
import numpy as np

RusLem = pymorphy2.MorphAnalyzer()

def bprint(l, sep = " "):
    print sep.join(l)

In [181]:
#sentences_normaliation
def normalization(collection):
    for i, text in enumerate(collection):
#         print text
        tokens = re.findall('[\w]+', text[0].strip().lower(), re.U)
        text[0] = " ".join([RusLem.parse(token)[0].normal_form for token in tokens])
    return collection

def get_tokens_probs(forward_index, token_list):
    tokens_cnt = 0
    token_probs = {token:0 for token in token_list}
    for sent_idx in forward_index.keys():
        for token in forward_index[sent_idx]:
            token_probs[token] += 1
        tokens_cnt += len(forward_index[sent_idx])
        
    token_probs = {token:token_probs[token] / float(tokens_cnt) for token in token_list}
    return token_probs
        
    
class NaiveBayesClassificator(object):
    def __init__(self):
        pass
    
    def train(self, collection_dir="./yandex_news/"):
        self.raw_collection = self.get_collection(collection_dir)
        self.collection = normalization(deepcopy(self.raw_collection))
        self.token_probs, self.class_probs, self.smoothing_values = self.get_tokens_probs(self.collection)
        return self
        
    def get_collection(self, collection_dir):
        """
        return list of pair [[text, class_id]
        sport - class 0
        policy - class 1
        """
        collection = []
        class_dirs = sorted(os.listdir(collection_dir))
        cur_class_id = 0
        for class_dir in class_dirs:
            if class_dir == 'sport':
                cur_class_id = 0
            elif class_dir == 'policy':
                cur_class_id = 1
            else:
                continue
                
            files = os.listdir(os.path.join(collection_dir, class_dir))
            
            for filename in files:
                with open(os.path.join(collection_dir, class_dir, filename)) as f:
                    f_text = " ".join(f.readlines()).decode('utf-8')
                    collection.append([f_text, cur_class_id])

        return collection
    
                      
    def get_tokens_probs(self, collection):
        """
        return {class_id:{token:token_freqs for this class}}
        """
        token_freqs = {}
        uniq_tokens = set()
        class_probs = {}
        for doc in self.collection:
            if doc[1] not in token_freqs:
                token_freqs[doc[1]] = {}
            if doc[1] not in class_probs:
                class_probs[doc[1]] = 0
                      
            class_probs[doc[1]] += 1
            for token in doc[0].split(" "):
#                 tokens = sent.split(" ")
                if token not in token_freqs[doc[1]]:
                    token_freqs[doc[1]][token] = 0
                    uniq_tokens.add(token)
                token_freqs[doc[1]][token] += 1
                      
        class_probs = {class_id: class_probs[class_id] / float(len(self.collection))
                       for class_id in class_probs.keys()}
        
        token_probs = {}
        smoothing_values = {}
        cnt_uniq_token = len(list(uniq_tokens))
        for class_id in token_freqs.keys():
            cnt_class_token = sum(token_freqs[class_id].values())
            smoothing_values[class_id] = 1. /  float(cnt_class_token + cnt_uniq_token)
            token_probs[class_id] = {token:\
                                     (token_freqs[class_id][token] + 1) / float(cnt_class_token + cnt_uniq_token)
                                    for token in token_freqs[class_id].keys()}
                      
        return token_probs, class_probs, smoothing_values 
                      
    def predict(self, doc):
        prob_doc = {class_id:1 for class_id in self.token_probs}
        doc_tokens = doc.split(" ")
        bprint(doc_tokens)
        for class_id in prob_doc.keys():
            prob_doc[class_id] = reduce(lambda x, y: x * y, [self.token_probs[class_id][token]
                                                             if token in self.token_probs[class_id]
                                                             else self.smoothing_values[class_id]\
                                                             for token in doc_tokens])
            print "Probability of class {}:{}".format(class_id, prob_doc[class_id])
        
            

def print_serp(rank, query, sent_cnt = 10):
    sorted_rank = sorted(rank.items(), key = operator.itemgetter(1), reverse=True)
    print "Query:",
    bprint([query])
    print "\n"
    serp = []
    
    for idx, pair in enumerate(sorted_rank):
#         print idx, pair
        if idx > sent_cnt:
            break
        print pair[1], "\t", pair[0], 
        bprint(collection.forward_index[pair[0]])
        print
        serp.append(pair[0])
    return serp

In [182]:
nb = NaiveBayesClassificator().train()

In [186]:
test_text_1 = u"Польский власть связал провокации КНДР с «планами России\
                .«С польской точки зрения действия Северной Кореи строго соотнесены и\
                связаны с российскими планами и агрессивным поведением России», \
                — приводит «Газета.ру» слова польского министра."
tokens = re.findall('[\w]+', test_text_1.strip().lower(), re.U)
test_text_1 = " ".join([RusLem.parse(token)[0].normal_form for token in tokens])
print test_text_1

польский власть связать провокация кндр с план россия с польский точка зрение действие северный корея строго соотнести и связать с российский план и агрессивный поведение россия приводить газета ру слово польский министр


In [187]:
nb.predict(test_text_1)
print "Значит, данный текст относится к классу политики"

польский власть связать провокация кндр с план россия с польский точка зрение действие северный корея строго соотнести и связать с российский план и агрессивный поведение россия приводить газета ру слово польский министр
Probability of class 0:5.47139928818e-91
Probability of class 1:3.96828026559e-88


In [189]:
test_text_2 = u"Анри может стать главным тренером сборной Уэльса. Экс-форвард «Арсенала» \
                Тьерри Анри может занять пост главного тренера сборной Уэльса. \
                Накануне уволен с данного места был Крис Коулмэн, перебравшийся в «Сандерленд». \
                Анри на данный момент занимает должность помощника главного тренера \
                сборной Бельгии Роберто Мартинеса."
tokens = re.findall('[\w]+', test_text_2.strip().lower(), re.U)
test_text_2 = " ".join([RusLem.parse(token)[0].normal_form for token in tokens])
print test_text_2

анри мочь стать главный тренер сборная уэльс экс форвард арсенал тьерри анри мочь занять пост главное тренер сборная уэльс накануне уволить с данный место быть крис коулмэн перебраться в сандерленд анри на данный момент занимать должность помощник главное тренер сборная бельгия роберто мартинес


In [190]:
nb.predict(test_text_2)
print "Значит, данный текст относится к классу спорт"

анри мочь стать главный тренер сборная уэльс экс форвард арсенал тьерри анри мочь занять пост главное тренер сборная уэльс накануне уволить с данный место быть крис коулмэн перебраться в сандерленд анри на данный момент занимать должность помощник главное тренер сборная бельгия роберто мартинес
Probability of class 0:4.61873125317e-122
Probability of class 1:7.49450887933e-127
Значит, данный текст относится к классу спорт
